In [ ]:
# ====================================================================
# 🌌 QEPC UNIVERSAL LOADER
# ====================================================================
import sys, os
# Add the project root (one level up) to the path so we can import the context
sys.path.append(os.path.abspath('..')) 

# Load everything from the central context file
from notebook_context import * # Trigger the success message
print_ready_message()
print("="*80)

In [ ]:
# --- Cell A: Load Data and Calculate Lambda ---
qa.qepc_step("Starting Advanced QEPC Prediction Engine...")

# 1. Load Data
schedule = qa.load_games()
team_stats = qa.load_team_stats() 

qa.qepc_step("Data Loaded. Calculating True ORtg/DRtg and Lambda...")

# 2. Calculate Lambda
schedule_with_lambda = compute_lambda(schedule, team_stats) 

# Diagnostic printout
lakers_ortg = team_stats[team_stats['Team'] == 'Los Angeles Lakers']['ORtg']
test_val = lakers_ortg.iloc[0] if not lakers_ortg.empty else "N/A"
print(f"Games found for simulation: {len(schedule_with_lambda)}")
print(f"Lakers Offensive Rating (Test): {test_val}")

In [ ]:
# --- Cell B: Run QEPC Simulation ---
qa.qepc_step("Running QEPC Monte Carlo Simulation (20,000 Trials)...")

predicted_schedule = run_qepc_simulation(schedule_with_lambda, num_trials=20000)

qa.qepc_step("Simulation Complete. Final Predictions Ready.")

In [ ]:
# --- Cell C: Show Final Predictions (Styled Output & Viz) ---

# 1. Prepare Data
viz_df = predicted_schedule.copy()
viz_df['Matchup'] = viz_df['Away Team'] + ' @ ' + viz_df['Home Team']
viz_df['Expected_Score_Total'] = viz_df['Sim_Home_Score'] + viz_df['Sim_Away_Score']
viz_df['Expected_Spread_Abs'] = viz_df['Expected_Spread'].abs()

# 2. Styled Table
styled_predictions = viz_df[[
    "Date", "Time", "Away Team", "Home Team", "Home_Win_Prob", "Away_Win_Prob", 
    "Expected_Spread", "Sim_Home_Score", "Sim_Away_Score"
]].head(10).copy()

styled_predictions['Home_Win_Prob (%)'] = (styled_predictions['Home_Win_Prob'] * 100).round(1)
styled_predictions['Away_Win_Prob (%)'] = (styled_predictions['Away_Win_Prob'] * 100).round(1)
styled_predictions.drop(columns=['Home_Win_Prob', 'Away_Win_Prob'], inplace=True)

qa.qepc_step("Top 10 Predictions (Styled Table)")

def color_win_prob(val):
    color = '#268bd2' if val > 65 else ('#5cb85c' if val > 55 else ('#f0ad4e' if val > 50 else '#d9534f'))
    return f'background-color: {color}; color: white' if val >= 50 else ''

styled_output = styled_predictions.style.applymap(
    color_win_prob, 
    subset=['Home_Win_Prob (%)', 'Away_Win_Prob (%)']
).format({
    'Expected_Spread': '{:+.2f}',
    'Sim_Home_Score': '{:.1f}',
    'Sim_Away_Score': '{:.1f}'
}).background_gradient(
    cmap='Blues', 
    subset=['Sim_Home_Score', 'Sim_Away_Score'], 
    vmin=90, vmax=130
)

display(styled_output)

# 3. Plotly Visualization
qa.qepc_step("Interactive Predictive Landscape (Plotly)")

viz_df['Favored_Team'] = np.where(viz_df['Expected_Spread'] > 0, viz_df['Home Team'], viz_df['Away Team'])

fig = px.scatter(
    viz_df, 
    x='Expected_Spread', 
    y='Expected_Score_Total', 
    size='Expected_Spread_Abs', 
    color='Favored_Team',
    hover_name='Matchup',
    hover_data={'Expected_Spread': ':.2f', 'Expected_Score_Total': ':.1f', 'Favored_Team': True},
    title='QEPC Predictive Landscape: Spread vs. Total (All Games)',
    template='plotly_dark' 
)

fig.add_vline(x=0, line_width=1, line_dash="dash", line_color="gray")
fig.show()

# ----------------------